

The two primary datasets we will be working with are both from the CDC: a survey on vaccination confidence, and a survey on depression and anxiety symptoms.




Processing:

In [ ]:
import pandas as pd
import numpy as np

Data on immunization confidence retrieved from
https://data.cdc.gov/Vaccinations/National-Immunization-Survey-Adult-COVID-Module-NI/udsf-9v7b
on 12/18/2021

Data on symptoms of depression and anxiety retrieved from
https://www.cdc.gov/nchs/covid19/pulse/mental-health.htm
on 12/18/2021